In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageDraw, ImageFont # Thêm thư viện PIL đầy đủ
import os
import random
import matplotlib.pyplot as plt
import numpy as np
print(f"Đã import các thư viện cần thiết")

Đã import các thư viện cần thiết


In [22]:
#định nghĩa parameters và cấu hình
#định nghĩa thiết bị
if torch.backends.mps.is_available():
    device=torch.device('mps')
elif torch.cude.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu')
print(f"Đang sử dụng thiết bị {device}")

#định nghĩa các tham số
batch_size=32
epochs=15
learning_rate=0.001
img_size=128
data_root_dir='./data'
print(f"Kích thước ảnh mục tiêu: {img_size}*{img_size}")
print(f"Batch size: {batch_size}")
print(f"Số lượng epochs: {epochs}")
print(f"Tốc độ học: {learning_rate}")
print(f"Thư mục gốc dữ liệu: {data_root_dir}")


Đang sử dụng thiết bị mps
Kích thước ảnh mục tiêu: 128*128
Batch size: 32
Số lượng epochs: 15
Tốc độ học: 0.001
Thư mục gốc dữ liệu: ./data


In [23]:
# xử lý đường dẫn dữ liệu và dataset
#định nghĩa các phép biến đổi
data_transform=transforms.Compose([
    transforms.Resize((img_size,img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
print(f"Đã định nghĩa các phép biến đổi dữ liệu")

#xử lý đường dẫn dữ liệu
print(f"Tiến hành xử lý đường dẫn dữ liệu và kiểm tra file ảnh")
train_images_pair=[]
test_images_pair=[]
data_valid=False

#đường dẫn đến data/train/watermarked và data/train/cleaned
train_cleaned_dir=os.path.join(data_root_dir,'train','cleaned')
train_watermarked_dir=os.path.join(data_root_dir,'train','watermarked')

 #kiểm tra xem đường dẫn có trỏ đến thư mục tồn tại
if not os.path.isdir(train_cleaned_dir):
    print(f"Thư mục {train_cleaned_dir} không tồn tại")
    data_valid=False
if not os.path.isdir(train_watermarked_dir):
    print(f"Thư mục {train_watermarked_dir} không tồn tại")
    data_valid=False

if data_valid:
    print(f"Đang quét thư mục {train_watermarked_dir}")
    try:
        train_files=sorted([
            f for f in os.listdir(train_watermarked_dir)
            if f.lower().endswith(('.png','.jpg','.jpeg'))
        ])
        if not train_files:
            print(f"Không tìm thấy file nào trong thư mục {train_watermarked_dir}")
        for file_name in train_files:
            watermarked_path=os.path.join(train_watermarked_dir,file_name)
            cleaned_path=os.path.join(train_cleaned_dir,file_name)
            if os.path.isfile(cleaned_path):
                train_images_pair.append((watermarked_path,cleaned_path))
            else:
                print(f"Bỏ qua {file_name} vì không tìm thấy ảnh sạch tương ứng")
        if not (train_images_pair):
            print(f"Không tạo được cặp ảnh hợp lệ cho tập train")
            data_valid=False
    except Exception as e:
        print(f"Lỗi khi quét thư mục {train_watermarked_dir}: {e}")
        data_valid=False
    
#xử lý tập val nếu tập train có tồn tại
#đường dẫn đến data/val/watermarked và data/val/cleaned
if data_valid:
    val_cleaned_dir=os.path.join(data_root_dir,'val','cleaned')
    val_watermarked_dir=os.path.join(data_root_dir,'val','watermarked')
#kiểm tra xem đường dẫn có trỏ đến thư mục tồn tại
if not os.path.isdir(val_cleaned_dir):
    print(f"Thư mục {val_cleaned_dir} không tồn tại")
    data_valid=False
if not os.path.isdir(val_watermarked_dir):
    print(f"Thư mục {val_watermarked_dir} không tồn tại")
    data_valid=False
if data_valid:
    try:
        print(f"Đang quét thư mục {val_watermarked_dir}")
        val_files=sorted([
            f for f in os.listdir(val_watermarked_dir)
            if f.lower().eandswwith(('.png','.jpg','.jpeg'))
        ])
        if not val_files:
            print(f"Không tìm thấy file nào trong thư mục {val_watermarked_dir}")
        for file_name in val_files:
            watermarked_path=os.path.join(val_watermarked_dir,file_name)
            cleaned_path=os.path.join(val_cleaned_dir,file_name)
            if os.path.isfile(cleaned_path):
                test_images_pair.append((watermarked_path,cleaned_path))
            else:
                print(f"Bỏ qua {file_name} vì không tìm thấy ảnh sạch tương ứng")
        if not (test_images_pair):
            print(f"Không tạo được tập ảnh hợp lệ cho tập val")
            data_valid=False
    except Exception as e:
        print(f"Lỗi khi quét thư mục {val_watermarked_dir}: {e}")
        data_valid=False

Đã định nghĩa các phép biến đổi dữ liệu
Tiến hành xử lý đường dẫn dữ liệu và kiểm tra file ảnh


NameError: name 'val_cleaned_dir' is not defined